In [65]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

In [66]:
df_athletes_activities = pd.read_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st1_df_athletes_activities.csv', parse_dates=['activity_date'])
df_athletes_activities

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,1,2020-07-09,6.03,39.0,6.47,9.28
1,1,2020-07-10,25.35,140.0,5.52,10.86
2,1,2020-07-12,7.57,55.0,7.27,8.26
3,1,2020-07-16,3.61,23.0,6.37,9.42
4,1,2020-07-20,25.66,127.0,4.95,12.12
...,...,...,...,...,...,...
903,7,2024-06-16,10.06,45.0,4.47,13.26
904,7,2024-06-18,5.20,33.0,6.35,9.27
905,7,2024-06-21,12.10,62.0,5.12,11.71
906,7,2024-06-26,4.08,22.0,5.39,11.13


In [67]:
df_athletes_activities_new = pd.DataFrame()

for id, df_grouped in df_athletes_activities.groupby('athlete_id'):

    df_grouped.insert(1, 'is_activity', 'Activity')

    # Data da primeira e última atividade para criar um range
    start, end = df_grouped['activity_date'].iloc[0], df_grouped['activity_date'].iloc[-1]

    # Dias entre a primeira e última atividade
    dates = pd.date_range(start, end, freq='d')

    # Colocando todas as datas em um df
    df_dates = pd.DataFrame({'activity_date': dates})

    # Right join pra poder manter os dados das atividades existentes mas adicionar linhas para os novos dias
    df_grouped = df_dates.merge(df_grouped, on='activity_date', how='left')

    # Preechendo os valores nulos como Dia sem atividade
    df_grouped['is_activity'].fillna('No Activity', inplace=True)
    
    # Preenchendo novamente a coluna id para evitar NaN das linhas sem atividade
    df_grouped['athlete_id'] = id

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities_new= pd.concat([df_athletes_activities_new, df_grouped])

df_athletes_activities_new['is_activity'] = (df_athletes_activities_new['is_activity'] == 'Activity') * 1
df_athletes_activities_new.reset_index(drop=True, inplace=True)
df_athletes_activities_new.fillna(0, inplace=True)
df_athletes_activities_new

,activity_date,athlete_id,is_activity,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2020-07-09,1,1,6.03,39.0,6.47,9.28
1,2020-07-10,1,1,25.35,140.0,5.52,10.86
2,2020-07-11,1,0,0.00,0.0,0.00,0.00
3,2020-07-12,1,1,7.57,55.0,7.27,8.26
4,2020-07-13,1,0,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...
4623,2024-06-24,7,0,0.00,0.0,0.00,0.00
4624,2024-06-25,7,0,0.00,0.0,0.00,0.00
4625,2024-06-26,7,1,4.08,22.0,5.39,11.13
4626,2024-06-27,7,0,0.00,0.0,0.00,0.00


In [68]:
# Número de atividades dentro desse período de dias em uma janela deslizante, por atleta
# Quantidade de atividades num período específico de dias
activity_days = 7

# Criação de feature para quantidade de atividades semanal
df_athletes_activities_new['weekly_activities_frequency'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['is_activity'].rolling(activity_days).sum()]
df_athletes_activities_new.dropna(subset= 'weekly_activities_frequency', inplace= True)
df_athletes_activities_new['weekly_activities_frequency'] = df_athletes_activities_new['weekly_activities_frequency'].astype(int)

# Criação de feature para quantidade de tempo semanal
df_athletes_activities_new['weekly_active_time'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(activity_days).sum()]
df_athletes_activities_new.dropna(subset= 'weekly_active_time', inplace= True)
df_athletes_activities_new['weekly_active_time'] = df_athletes_activities_new['weekly_active_time'].astype(int)

# Criação de feature para distância percorrida na semana
df_athletes_activities_new['weekly_dist_traveled'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(activity_days).sum()]
df_athletes_activities_new.dropna(subset= 'weekly_dist_traveled', inplace= True)
df_athletes_activities_new['weekly_dist_traveled'] = df_athletes_activities_new['weekly_dist_traveled'].astype(int)

In [69]:
# Dropando as linhas que não atividades
df_athletes_activities_new.drop(df_athletes_activities_new[df_athletes_activities_new['is_activity'] == 0].index, axis=0, inplace=True)
# Dropando as coluna que checam se é ou não atividade e se é ou não corrida (não precisa mais)
df_athletes_activities_new.drop(['is_activity'], axis=1, inplace= True)
# Resetando index depois do drop
df_athletes_activities_new.reset_index(drop=True, inplace=True)

In [70]:
# Fazendo a diferença entre a atividade seguinte e a anterior e preenchendo a primeira linha com 0
df_athletes_activities_new['days_since_last_activity'] = (df_athletes_activities_new.groupby('athlete_id')['activity_date'].diff().dt.days.fillna(0)).astype(int)

In [71]:
df_athletes_activities_new['will_churn'] = (((df_athletes_activities_new['days_since_last_activity'] >= 14) * 1).shift(-1).fillna(0)).astype(int)
df_athletes_activities_new

,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),weekly_activities_frequency,weekly_active_time,weekly_dist_traveled,days_since_last_activity,will_churn
0,2020-07-28,1,13.22,92.0,6.96,8.62,4,314,48,0,0
1,2020-08-04,1,19.22,85.0,4.42,13.57,1,85,19,7,0
2,2020-08-06,1,25.69,128.0,4.98,12.04,2,213,44,2,0
3,2020-08-11,1,25.60,119.0,4.65,12.91,2,247,51,5,0
4,2020-08-13,1,25.31,131.0,5.18,11.59,2,250,50,2,0
...,...,...,...,...,...,...,...,...,...,...,...
854,2024-06-16,7,10.06,45.0,4.47,13.26,3,116,22,3,0
855,2024-06-18,7,5.20,33.0,6.35,9.27,3,108,20,2,0
856,2024-06-21,7,12.10,62.0,5.12,11.71,3,140,27,3,0
857,2024-06-26,7,4.08,22.0,5.39,11.13,2,84,16,5,0


In [72]:
df_athletes_activities_new['will_churn'].value_counts(normalize=True)

will_churn
0    0.934808
1    0.065192
Name: proportion, dtype: float64

In [73]:
df_new_features = pd.DataFrame()

for atleta, dados_atleta in df_athletes_activities_new.groupby('athlete_id'):
    dados_atleta['athlete_max_dist'] = dados_atleta['total_distance (km)'].max()
    dados_atleta['athlete_avg_dist'] = round(dados_atleta['total_distance (km)'].mean(), 2)

    dados_atleta['athlete_max_time'] = dados_atleta['total_time (min)'].max()
    dados_atleta['athlete_avg_time'] = round(dados_atleta['total_time (min)'].mean(), 2)

    dados_atleta['athlete_max_pace'] = dados_atleta['pace (min/km)'].max()
    dados_atleta['athlete_avg_pace'] = round(dados_atleta['pace (min/km)'].mean(), 2)

    dados_atleta['athlete_max_speed'] = dados_atleta['speed (km/h)'].max()  
    dados_atleta['athlete_avg_speed'] = round(dados_atleta['speed (km/h)'].mean(), 2)
    
    dados_atleta['binary_reincidence'] = (dados_atleta['will_churn'].shift(1).cumsum() > 0) * 1 
    dados_atleta['accumulated_reincidence'] = (dados_atleta['will_churn'].shift(1).cumsum().fillna(0)).astype(int)

    df_new_features = pd.concat([df_new_features, dados_atleta])

df_new_features


,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),weekly_activities_frequency,weekly_active_time,weekly_dist_traveled,days_since_last_activity,...,athlete_max_dist,athlete_avg_dist,athlete_max_time,athlete_avg_time,athlete_max_pace,athlete_avg_pace,athlete_max_speed,athlete_avg_speed,binary_reincidence,accumulated_reincidence
0,2020-07-28,1,13.22,92.0,6.96,8.62,4,314,48,0,...,85.73,24.40,352.0,104.61,9.76,5.21,22.13,12.64,0,0
1,2020-08-04,1,19.22,85.0,4.42,13.57,1,85,19,7,...,85.73,24.40,352.0,104.61,9.76,5.21,22.13,12.64,0,0
2,2020-08-06,1,25.69,128.0,4.98,12.04,2,213,44,2,...,85.73,24.40,352.0,104.61,9.76,5.21,22.13,12.64,0,0
3,2020-08-11,1,25.60,119.0,4.65,12.91,2,247,51,5,...,85.73,24.40,352.0,104.61,9.76,5.21,22.13,12.64,0,0
4,2020-08-13,1,25.31,131.0,5.18,11.59,2,250,50,2,...,85.73,24.40,352.0,104.61,9.76,5.21,22.13,12.64,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,2024-06-16,7,10.06,45.0,4.47,13.26,3,116,22,3,...,13.24,8.42,70.0,45.14,9.23,5.43,13.77,11.12,0,0
855,2024-06-18,7,5.20,33.0,6.35,9.27,3,108,20,2,...,13.24,8.42,70.0,45.14,9.23,5.43,13.77,11.12,0,0
856,2024-06-21,7,12.10,62.0,5.12,11.71,3,140,27,3,...,13.24,8.42,70.0,45.14,9.23,5.43,13.77,11.12,0,0
857,2024-06-26,7,4.08,22.0,5.39,11.13,2,84,16,5,...,13.24,8.42,70.0,45.14,9.23,5.43,13.77,11.12,0,0


In [74]:
df_new_features.to_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st2_df_athletes_activities.csv', index = False)